In [1]:
import requests
from tqdm.auto import tqdm
from datetime import datetime
import pandas as pd


TOKEN = "378040d4378040d4378040d40337f73b4133780378040d457fdc4b7843740eb0ed5954b"
VERSION = "5.131"


Находим _id_ всех стран:

In [2]:
get_countries = "https://api.vk.com/method/database.getCountries"


In [3]:
countries_data = requests.get(
    get_countries, 
    params={
        "need_all": '1', 
        "count": '1000', 
        "v": VERSION, 
        "access_token": TOKEN  
    }
).json()['response']['items']


Находим _id_ стран СНГ (для РФ - _id: 1_):

In [5]:
cis_countries = ['Azerbaijan', 'Armenia', 'Belarus', 'Georgia', 'Kazakhstan', 'Kyrgyzstan', 'Moldova', 'Tajikistan', 'Uzbekistan', 'Turkmenistan', 'Ukraine']
baltics_countries = ['Latvia', 'Lithuania', 'Estonia']


In [6]:
cis_countries_ids = []
for cis_country in cis_countries:
    for country in countries_data:
        if country['title'] == cis_country:
            cis_countries_ids.append(country['id'])

baltics_countries_ids = []
for baltics_country in baltics_countries:
    for country in countries_data:
        if country['title'] == baltics_country:
            baltics_countries_ids.append(country['id'])   
            

In [8]:
cis_countries_ids


[5, 6, 3, 7, 4, 11, 15, 16, 18, 17, 2]

Находим все _id_ городов, являющихся пунктами в маршрутах, создаем список с уникальными _id_:

In [ ]:
df = pd.read_csv('all_routes_fm.csv', sep='\t')
routes = list(df['route'])
del routes[-1]


In [ ]:
ids = []

for route in routes:
    _ = route.split(', ')
    ids.extend(_)
    

In [ ]:
ids = list(set(ids))
len(ids)

idds = []
for i in ids:
    ii = int(i)
    idds.append(ii)
    

Находим названия городов по списку с _id_:

In [ ]:
get_cities_by_id = "https://api.vk.com/method/database.getCitiesById"


In [ ]:
with open('cities_ids.txt', 'a', encoding='utf-8') as city_id:
    for ct in idds:
        ctitles = requests.get(
            get_cities_by_id, 
            params={
                "city_ids": ct,
                "v": VERSION, 
                "access_token": TOKEN  
            }
        ).json()['response']
        if ctitles:
            city_id.write(str(ctitles[0]['id']) + ',' + ctitles[0]['title'] + '\n')
    

Создаем словари вида _{'id': int, 'title': str}_:

In [ ]:
city_paires_id_title = []
with open('cities_ids.txt', 'r', encoding='utf-8') as fucking_shit:
    pairs = fucking_shit.read().split('\n')
    for pair in pairs:
        if pair:
            p = pair.split(',')
            if p[1]: # некоторые города были удалены, поэтому названий у них нет, но id остался
                city_paires_id_title.append({'id': int(p[0]), 'title': p[1]})
            

In [ ]:
len(city_paires_id_title)


Создаем файл для каждой страны со списком городов (строка - _id,title_):

In [9]:
get_cities = "https://api.vk.com/method/database.getCities"


In [ ]:
all_countries = ['Russia']
all_countries.extend(cis_countries)
all_countries.extend(baltics_countries)

all_countries_ids = [1]
all_countries_ids.extend(cis_countries_ids)
all_countries_ids.extend(baltics_countries_ids)

In [ ]:
for c in range(len(all_countries)):
    i = 1000
    j = 0
    with open(all_countries[c] + '.txt', 'a', encoding='utf-8') as country_file:
        a = []
        while True:
            c_ = requests.get(
                get_cities, 
                params={
                    "country_id": all_countries_ids[c], 
                    "count": i,
                    "offset": j,
                    "v": VERSION, 
                    "need_all": 1,
                    "access_token": TOKEN  
                }
            ).json()['response']['items']
            a.extend(c_)
            for each in c_:
                country_file.write(str(each['id'])+','+each['title']+'\n')
            j+=1000
    